In [1]:
user_agent = ''
# Type your user agent in ""
# your user agent can be found here: https://www.whatismybrowser.com/detect/what-is-my-user-agent/

In [2]:
from bs4 import BeautifulSoup as bs
from collections import Counter
from konlpy.tag import Hannanum
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import aiohttp
import asyncio
import random
import requests
import time
import re
import matplotlib.pyplot as plt

In [ ]:
global articles

def scrape_page(i):
    pg = str(i)
    time.sleep(random.uniform(1, 5))
    try:
        with requests.Session() as session:
            res = session.get(f"https://cafe.naver.com/ArticleList.nhn?search.clubid=11262350&search.boardtype=L&search.totalCount=151&search.cafeId=11262350&search.page={pg}", headers={"User-Agent": user_agent})
            if res.status_code == 200:
                soup = bs(res.text, "html.parser")
                articles = []
                parsed_datas = soup.find_all("a", {"class": "article"})
                if not parsed_datas:
                    return f"No articles found on page {pg}"
                for data in parsed_datas:
                    article_text = str(data).strip().replace("\n", " ").replace("    ", " ")
                    article_contents = re.findall(r'</span>(.*?)</a>', article_text)
                    if article_contents:
                        articles.append(article_contents[0].strip())
                return articles
            else:
                return []
    except Exception as e:
        return []

articles = []
cnt = 0
with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
    for result in tqdm(executor.map(scrape_page, range(1, 1150000)), total=1150000):
        cnt += 1
        if isinstance(result, list):
            articles.extend(result)
            if cnt % 10000 == 0:
                with open('text'+str(cnt//10000)+'.txt', 'w', encoding='utf-8') as file:
                    for article in articles:
                        file.write(article + '\n')
                articles = []
        else:
            print(result)

# Saving the articles to a file after collecting

print('done')

  0%|                                                                        | 240/1150000 [08:09<285:38:40,  1.12it/s]

In [ ]:
hannanum = Hannanum()
words = []

for article in all_articles:
    nouns = hannanum.nouns(article)
    words+=nouns

print(len(words))

In [ ]:
counter = Counter(all_articles)

In [ ]:
from wordcloud import WordCloud

In [ ]:
img = WordCloud(font_path = r"", # Type your font's location
    background_color="white",
    height = 1000,
    width = 1000).generate_from_frequencies(counter)
plt.imshow(img)